[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PetiteIA/schema_mechanism/blob/master/experiments/smell_environment.ipynb)

# THE SMELL ENVIRONMENT

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from ipywidgets import Button, HBox,VBox, Output
from IPython.display import display

## Define the actions

In [2]:
FORWARD = 0
TURN_LEFT = 1
TURN_RIGHT = 2
FEEL_FRONT = 3
FEEL_LEFT = 4
FEEL_RIGHT = 5

## Define the feedbacks

In [44]:
STABLE = 0
BUMP = 1
INCREASE_LEFT = 2
INCREASE_RIGHT = 3
INCREASE_FRONT = 4
DECREASE = 5
EAT = 6

## Define the environment 

Choose the palette with the help of https://coolors.co/palettes/trending

In [4]:
agent_color = "#1976D2"
colors = ["#D6D6D6",   '#5C946E',  '#FAE2DB',        '#535865',      "#F93943", "#E365C1"]

In [5]:
save_dir = "sav"

# Directions
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
# Display codes
FEELING_EMPTY = 2
FEELING_WALL = 3
BUMPING = 4
TARGET = 5

In [45]:
class SmallLoop():
    def __init__(self, position, direction):
        self.grid = np.array([
            [1, 1, 1, 1, 1, 1], 
            [1, 0, 0, 0, 0, 1],
            [1, 0, 0, 5, 0, 1],
            [1, 0, 0, 0, 0, 1],
            [1, 0, 0, 0, 0, 1],
            [1, 1, 1, 1, 1, 1]
        ])
        self.maze = self.grid.copy()
        self.position = np.array(position)  # Using NumPy array of shape (2)
        self.direction = direction
        self.cmap = ListedColormap(colors)
        self.norm = BoundaryNorm([-0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5], self.cmap.N)
        self.marker_size = 400
        self.marker_map = {LEFT: '<', DOWN: 'v', RIGHT: '>', UP: '^'}
        self.marker_color = agent_color
        self.directions = np.array([
            [0, -1],  # Left
            [1, 0],   # Down
            [0, 1],   # Right
            [-1, 0]   # Up
        ])
        self.previous_smell = 0
        self.smell_feedback = np.array([
            [STABLE,   INCREASE_LEFT, INCREASE_RIGHT, INCREASE_FRONT], 
            [DECREASE, STABLE,        STABLE,         INCREASE_FRONT], 
            [DECREASE, STABLE,        STABLE,         INCREASE_FRONT], 
            [DECREASE, DECREASE,      DECREASE,       INCREASE_FRONT] 
        ])
        

    def smell(self):
        """Return the smell feedback: 1: smell left, 2: smell right, 3: smell front"""
        smell_up_left = np.any(self.grid[:self.position[0]+1, :self.position[1]+1] == TARGET) 
        smell_up_right = np.any(self.grid[:self.position[0]+1, self.position[1]:] == TARGET) 
        smell_down_left = np.any(self.grid[self.position[0]:, :self.position[1]+1] == TARGET) 
        smell_down_right = np.any(self.grid[self.position[0]:, self.position[1]:] == TARGET) 
        print(f"Smell up-left:{smell_up_left}, up-right:{smell_up_right}, down-left:{smell_down_left}, down-right:{smell_down_right}")
        smell_left = {LEFT: smell_down_left, DOWN: smell_down_right, RIGHT: smell_up_right, UP: smell_up_left}[self.direction]
        smell_right = {LEFT: smell_up_left, DOWN: smell_down_left, RIGHT: smell_down_right, UP: smell_up_right}[self.direction]
        smell = smell_left + 2 * smell_right 
        result = self.smell_feedback[self.previous_smell, smell]
        self.previous_smell = smell
        return result
    
    def outcome(self, action):
        """Update the grid. Return the outcome of the action."""
        result = STABLE

        if action == FORWARD:  
            target_position = self.position + self.directions[self.direction]
            if self.grid[tuple(target_position)] in [0, TARGET]:  # Don't bump in targets
                self.position[:] = target_position
            else:
                result = BUMP
                self.maze[tuple(target_position)] = BUMPING
        
        elif action == TURN_RIGHT:
            self.direction = {LEFT: UP, DOWN: LEFT, RIGHT: DOWN, UP: RIGHT}[self.direction]
        
        elif action == TURN_LEFT:
            self.direction = {LEFT: DOWN, DOWN: RIGHT, RIGHT: UP, UP: LEFT}[self.direction]
        
        elif action == FEEL_FRONT:
            feeling_position = self.position + self.directions[self.direction]
            if self.grid[tuple(feeling_position)] == 0:
                self.maze[tuple(feeling_position)] = FEELING_EMPTY
            else:
                result = BUMP
                self.maze[tuple(feeling_position)] = FEELING_WALL
        
        elif action == FEEL_LEFT:
            feeling_position = self.position + self.directions[(self.direction + 1) % 4]
            if self.grid[tuple(feeling_position)] == 0:
                self.maze[tuple(feeling_position)] = FEELING_EMPTY
            else:
                result = BUMP
                self.maze[tuple(feeling_position)] = FEELING_WALL
        
        elif action == FEEL_RIGHT:
            feeling_position = self.position + self.directions[self.direction - 1]
            if self.grid[tuple(feeling_position)] == 0:
                self.maze[tuple(feeling_position)] = FEELING_EMPTY
            else:
                result = BUMP
                self.maze[tuple(feeling_position)] = FEELING_WALL

        # Smell
        smell_feedback = self.smell()
        if self.grid[self.position[0], self.position[1]] == TARGET:
            self.grid[self.position[0], self.position[1]] = 0
            self.maze[self.position[0], self.position[1]] = 0
            result = EAT
        # If not bump then smell_feeback
        if not result:
            result = smell_feedback
        
        print(f"Line: {self.position[0]}, Column: {self.position[1]}, direction: {self.direction}")
        return result  
    
    def display(self):
        """Display the grid in the notebook"""
        out.clear_output(wait=True)
        with out:
            fig, ax = plt.subplots()
            ax.imshow(self.maze, cmap=self.cmap, norm=self.norm)
            plt.scatter(self.position[1], self.position[0], s=self.marker_size, marker=self.marker_map[self.direction], c=self.marker_color)
            plt.show()
    
    def save(self, step):
        """Save the display as a PNG file"""
        fig, ax = plt.subplots()
        ax.set_xticks([])
        ax.set_yticks([])
        ax.axis('off')
        ax.imshow(self.maze, cmap=self.cmap, norm=self.norm)
        plt.scatter(self.position[1], self.position[0], s=self.marker_size, marker=self.marker_map[self.direction], c=self.marker_color)
        ax.text(4.5, 0, f"{step:>3}", fontsize=12, color='White')
        plt.savefig(f"{save_dir}/{step:03}.png", bbox_inches='tight', pad_inches=0, transparent=True)
        plt.close(fig)
    
    def clear(self, clear):
        """Clear the grid display"""
        if clear:
            self.maze[:, :] = self.grid


## Display the environment

In [46]:
small_loop = SmallLoop([1, 1], 0)

Create the control buttons

In [47]:
def forward(change):
    small_loop.outcome(FORWARD)
    small_loop.display()
    small_loop.clear(True)

def turn_left(change):
    small_loop.outcome(TURN_LEFT)
    small_loop.display()
    small_loop.clear(True)

def turn_right(change):
    small_loop.outcome(TURN_RIGHT)
    small_loop.display()
    small_loop.clear(True)

def feel_left(change):
    small_loop.outcome(FEEL_LEFT)
    small_loop.display()
    small_loop.clear(True)

def feel_right(change):
    small_loop.outcome(FEEL_RIGHT)
    small_loop.display()
    small_loop.clear(True)

def feel_front(change):
    small_loop.outcome(FEEL_FRONT)
    small_loop.display()
    small_loop.clear(True)

def save(change):
    small_loop.save(0)

def target(change):
    small_loop.grid[TARGET_L, TARGET_C] = TARGET
    small_loop.maze[TARGET_L, TARGET_C] = TARGET
    small_loop.display()
    small_loop.clear(True)
    

# Create the buttons
forward_button = Button(description="Forward")
left_button = Button(description="Turn left")
right_button = Button(description="Turn right")
feel_left_button = Button(description="Feel left")
feel_front_button = Button(description="Feel Front")
feel_right_button = Button(description="Feel right")
print_button = Button(description="Print")
target_button = Button(description="target")

# Bind button clicks to functions
forward_button.on_click(forward)
left_button.on_click(turn_left)
right_button.on_click(turn_right)
feel_left_button.on_click(feel_left)
feel_right_button.on_click(feel_right)
feel_front_button.on_click(feel_front)
print_button.on_click(save)
target_button.on_click(target)

# Create the control bar
buttons = HBox([forward_button, left_button, right_button, feel_front_button, feel_left_button, feel_right_button, target_button, print_button])

In [48]:
TARGET_L = 3
TARGET_C = 2

In [49]:
out = Output()
small_loop.display()
display(out)
display(buttons)

Output()

## Enact actions

Use `Ctrl+Enter` to run the cell below and stay on it.

In [56]:
result = small_loop.outcome(FORWARD) # 0
small_loop.display()
print(f"outcome: {result}") 

Smell up-left:False, up-right:False, down-left:False, down-right:True
Line: 1, Column: 1, direction: 0
outcome: 1


In [57]:
result = small_loop.outcome(TURN_LEFT) # 1
small_loop.display()
print(f"outcome: {result}") 

Smell up-left:False, up-right:False, down-left:False, down-right:True
Line: 1, Column: 1, direction: 1
outcome: 2


In [52]:
result = small_loop.outcome(TURN_RIGHT) # 2
small_loop.display()
print(f"outcome: {result}") 

Smell up-left:False, up-right:False, down-left:False, down-right:True
Line: 1, Column: 1, direction: 0
outcome: 5


In [53]:
result = small_loop.outcome(FEEL_FRONT) # 3
small_loop.display()
print(f"outcome: {result}") 

Smell up-left:False, up-right:False, down-left:False, down-right:True
Line: 1, Column: 1, direction: 0
outcome: 1


In [54]:
result = small_loop.outcome(FEEL_LEFT) # 4
small_loop.display()
print(f"outcome: {result}") 

Smell up-left:False, up-right:False, down-left:False, down-right:True
Line: 1, Column: 1, direction: 0
outcome: 0


In [53]:
result = small_loop.outcome(FEEL_RIGHT) # 5
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 0
outcome: 1
